## Entitás adathalmaz

### Betöltés

In [4]:
from datasets import load_dataset

dataset = load_dataset('boapps/kmdb_classification')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3618 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/43060 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6598 [00:00<?, ? examples/s]

In [2]:
import spacy
spacy.require_gpu()

True

In [3]:
import hu_core_news_trf

nlp = hu_core_news_trf.load()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/boa/.conda/envs/ai/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so.11.0'), PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'c

### Feldolgozás

In [4]:
entity_dataset = dataset.filter(lambda article: article['label'] == 1)

Loading cached processed dataset at /home/boa/szakdoga/datasets/base/classification_dataset/train/cache-1bbf6c3ab28d4a59.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/base/classification_dataset/validation/cache-782105f591e71bb6.arrow
Loading cached processed dataset at /home/boa/szakdoga/datasets/base/classification_dataset/test/cache-bb36ead512c3d02e.arrow


In [5]:
def get_entities(articles):
    results = {'entities': []}
    try:
        for doc in nlp.pipe(articles['text'], disable=["tagger", "parser", "lemma_smoother"], ):
            results['entities'].append([(ent.lemma_, ent.label_) for ent in doc.ents])
    except Exception:
        results['entities'].append(None)
    return results

entity_dataset = entity_dataset.map(get_entities, batched=True, batch_size=1)

Map:   0%|          | 0/26661 [00:00<?, ? examples/s]

Map:   0%|          | 0/3410 [00:00<?, ? examples/s]

Map:   0%|          | 0/1720 [00:00<?, ? examples/s]

In [6]:
entity_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections', 'entities'],
        num_rows: 26661
    })
    validation: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections', 'entities'],
        num_rows: 3410
    })
    test: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections', 'entities'],
        num_rows: 1720
    })
})

In [7]:
entity_dataset.save_to_disk('datasets/intermediate/entity_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/26661 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3410 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1720 [00:00<?, ? examples/s]